In [16]:
import herringbone as hb

In [ ]:

# create an MDP
map_names = ["-", "example", "easy", "danger_holes", "double_fish", "wall_of_death", "example2", "mega"]
selected_map_id = 5

state_path = "herringbone/env_core/config/state_config.json"
map_path = f"herringbone/env_core/maps/{map_names[selected_map_id]}.csv"
action_path = "herringbone/env_core/config/action_config.json"

demo_mdp = hb.MDP(state_path, map_path, action_path, seed=42)


In [18]:

random_policy = hb.EpsilonGreedyPolicy(mdp=demo_mdp, epsilon=1)
episode = hb.Episode(mdp=demo_mdp, policy=random_policy, max_depth=1000)
episode.peek()
episode.run("sar")
print(episode.trajectory)

╔═════════╦═════════╦═════════╦═════════╦═════════╗
║         ║         ║         ║         ║ <x)))>< ║
╠═════════╬═════════╬═════════╬═════════╬═════════╣
║         ║         ║         ║         ║         ║
╠═════════╬═════════╬═════════╬═════════╬═════════╣
║         ║         ║         ║         ║         ║
╠═════════╬═════════╬═════════╬═════════╬═════════╣
║         ║         ║         ║         ║         ║
╠═════════╬═════════╬═════════╬═════════╬═════════╣
║         ║         ║         ║         ║ <x)))>< ║
╚═════════╩═════════╩═════════╩═════════╩═════════╝
t: 0 | S: [0, 0], R: nan, A: ↑
t: 1 | S: [0, 0], R: -1, A: ↓
t: 2 | S: [1, 0], R: -1, A: ↑
t: 3 | S: [0, 0], R: -1, A: ↑
t: 4 | S: [0, 0], R: -1, A: ↑
t: 5 | S: [0, 0], R: -1, A: ↓
t: 6 | S: [1, 0], R: -1, A: ↑
t: 7 | S: [0, 0], R: -1, A: →
t: 8 | S: [0, 1], R: -1, A: ←
t: 9 | S: [0, 0], R: -1, A: ↑
t: 10 | S: [0, 0], R: -1, A: ↓
t: 11 | S: [1, 0], R: -1, A: ←
t: 12 | S: [1, 0], R: -1, A: ↓
t: 13 | S: [2, 0], R: -1, A: ←
t: 

In [19]:
# Add the render function for previewing a Value Function (hate this!)
def preview_V(mdp, learned_V):
    
    states_2d = mdp.get_board().states
    x = len(states_2d)
    y = len(states_2d[0])
    
    v_values =  ['%.2f' % v for v in list(learned_V.values())]

    two_d_list = [v_values[i * y:(i + 1) * y] for i in range(x)]

    # Print 2D list
    for row in two_d_list:
        print(row)

In [20]:
policy = hb.EpsilonGreedyPolicy(mdp=demo_mdp, epsilon=1)  # defaults to uniform
print(policy)

╔═════════╦═════════╦═════════╦═════════╦═════════╗
║ ↑/↓/←/→ ║ ↑/↓/←/→ ║ ↑/↓/←/→ ║ ↑/↓/←/→ ║ ↑/↓/←/→ ║
╠═════════╬═════════╬═════════╬═════════╬═════════╣
║ ↑/↓/←/→ ║ ↑/↓/←/→ ║ ↑/↓/←/→ ║ ↑/↓/←/→ ║ ↑/↓/←/→ ║
╠═════════╬═════════╬═════════╬═════════╬═════════╣
║ ↑/↓/←/→ ║ ↑/↓/←/→ ║ ↑/↓/←/→ ║ ↑/↓/←/→ ║ ↑/↓/←/→ ║
╠═════════╬═════════╬═════════╬═════════╬═════════╣
║ ↑/↓/←/→ ║ ↑/↓/←/→ ║ ↑/↓/←/→ ║ ↑/↓/←/→ ║ ↑/↓/←/→ ║
╠═════════╬═════════╬═════════╬═════════╬═════════╣
║ ↑/↓/←/→ ║ ↑/↓/←/→ ║ ↑/↓/←/→ ║ ↑/↓/←/→ ║ ↑/↓/←/→ ║
╚═════════╩═════════╩═════════╩═════════╩═════════╝


In [21]:
episode = hb.Episode(mdp=demo_mdp, policy=policy, max_depth=1000)

In [22]:
episode.run("sar")

t: 0 | S: [0, 0], R: nan, A: ↑
t: 1 | S: [0, 0], R: -1, A: ↑
t: 2 | S: [0, 0], R: -1, A: ↓
t: 3 | S: [1, 0], R: -1, A: →
t: 4 | S: [1, 1], R: -1, A: →
t: 5 | S: [1, 2], R: -1, A: →
t: 6 | S: [1, 3], R: -1, A: ↑
t: 7 | S: [0, 3], R: -1, A: ↑
t: 8 | S: [0, 3], R: -1, A: ←
t: 9 | S: [0, 2], R: -1, A: ←
t: 10 | S: [0, 1], R: -1, A: →
t: 11 | S: [0, 2], R: -1, A: ↑
t: 12 | S: [0, 2], R: -1, A: ←
t: 13 | S: [0, 1], R: -1, A: ↓
t: 14 | S: [1, 1], R: -1, A: ↑
t: 15 | S: [0, 1], R: -1, A: ←
t: 16 | S: [0, 0], R: -1, A: ↓
t: 17 | S: [1, 0], R: -1, A: ↓
t: 18 | S: [2, 0], R: -1, A: ↑
t: 19 | S: [1, 0], R: -1, A: →
t: 20 | S: [1, 1], R: -1, A: ←
t: 21 | S: [1, 0], R: -1, A: ←
t: 22 | S: [1, 0], R: -1, A: ↓
t: 23 | S: [2, 0], R: -1, A: ↑
t: 24 | S: [1, 0], R: -1, A: →
t: 25 | S: [1, 1], R: -1, A: ↓
t: 26 | S: [2, 1], R: -1, A: →
t: 27 | S: [2, 2], R: -1, A: ↓
t: 28 | S: [3, 2], R: -1, A: →
t: 29 | S: [3, 3], R: -1, A: ↓
t: 30 | S: [4, 3], R: -1, A: →


In [23]:
DISCOUNT = 0.9

In [24]:
N = 1000
mc_predictor = hb.MonteCarloPredictor(demo_mdp, discount=DISCOUNT)
mc_predictor.evaluate_policy(policy, n_samples=N)

In [25]:
preview_V(demo_mdp, mc_predictor.value_functions)

['-8.19', '-7.34', '-4.93', '0.33', '0.00']
['-8.26', '-7.52', '-6.01', '-2.38', '1.95']
['-8.32', '-7.61', '-6.47', '-3.98', '-1.65']
['-8.40', '-7.48', '-5.97', '-3.02', '0.64']
['-8.15', '-7.41', '-4.95', '-0.06', '0.00']


In [26]:

N = 1000
mc_control = hb.MonteCarloController(demo_mdp, discount=DISCOUNT, epsilon=0.25, start_coords=(0,0))
mc_control.train(n_episodes=N)
trained_policy = mc_control.policy

In [27]:
print(trained_policy.get_policy())
print(trained_policy)

{[0, 0]: {↑: 0.0625, ↓: 0.0625, ←: 0.0625, →: 0.8125}, [0, 1]: {↑: 0.0625, ↓: 0.0625, ←: 0.0625, →: 0.8125}, [0, 2]: {↑: 0.0625, ↓: 0.0625, ←: 0.0625, →: 0.8125}, [0, 3]: {↑: 0.0625, ↓: 0.0625, ←: 0.0625, →: 0.8125}, [0, 4]: {↑: 0.25, ↓: 0.25, ←: 0.25, →: 0.25}, [1, 0]: {↑: 0.0625, ↓: 0.0625, ←: 0.0625, →: 0.8125}, [1, 1]: {↑: 0.0625, ↓: 0.0625, ←: 0.0625, →: 0.8125}, [1, 2]: {↑: 0.0625, ↓: 0.0625, ←: 0.0625, →: 0.8125}, [1, 3]: {↑: 0.0625, ↓: 0.0625, ←: 0.0625, →: 0.8125}, [1, 4]: {↑: 0.8125, ↓: 0.0625, ←: 0.0625, →: 0.0625}, [2, 0]: {↑: 0.8125, ↓: 0.06250000000000001, ←: 0.0625, →: 0.0625}, [2, 1]: {↑: 0.8125, ↓: 0.0625, ←: 0.0625, →: 0.0625}, [2, 2]: {↑: 0.0625, ↓: 0.0625, ←: 0.0625, →: 0.8125}, [2, 3]: {↑: 0.8125, ↓: 0.0625, ←: 0.0625, →: 0.0625}, [2, 4]: {↑: 0.8125, ↓: 0.0625, ←: 0.0625, →: 0.0625}, [3, 0]: {↑: 0.06388748636843884, ↓: 0.06267386314987172, ←: 0.06245100490157998, →: 0.8109876455801094}, [3, 1]: {↑: 0.8124977430218238, ↓: 0.06250193894588918, ←: 0.06250035344079531,

In [28]:
episode = hb.Episode(mdp=demo_mdp, policy=trained_policy, max_depth=1000)
episode.run("sar")

t: 0 | S: [0, 0], R: nan, A: →
t: 1 | S: [0, 1], R: -1, A: →
t: 2 | S: [0, 2], R: -1, A: →
t: 3 | S: [0, 3], R: -1, A: →


In [29]:
N = 1000
mc_predictor = hb.MonteCarloPredictor(demo_mdp, discount=DISCOUNT)
mc_predictor.evaluate_policy(trained_policy, n_samples=N)

In [30]:
preview_V(demo_mdp, mc_predictor.value_functions)

['2.56', '4.40', '6.67', '9.28', '0.00']
['1.80', '2.88', '4.85', '6.92', '9.21']
['0.06', '1.33', '2.90', '4.60', '7.03']
['-1.39', '-0.43', '3.12', '3.02', '10.00']
['0.00', '0.00', '0.00', '0.63', '0.00']
